## Kernel to load: vax_inc_general

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import pycountry
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import resample
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import os
import optuna 
import logging
import time
from sklearn.model_selection import cross_val_score
import random
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

In [2]:
notebook_dir = os.path.dirname(os.getcwd())
source_data_path=os.path.join(notebook_dir, "Common Source Data")

In [3]:
country_ghd_region=dict()

In [4]:
regions = {
    "Andean Latin America": [
        "Bolivia", "Ecuador", "Peru"
    ],
    "Australasia": [
        "Australia", "New Zealand", "American Samoa", "Cook Islands", "Tokelau"
    ],
    "Caribbean": [
        "Antigua and Barbuda", "Bahamas", "Barbados", "Belize", "Cuba", "Dominica",
        "Dominican Republic", "Grenada", "Guyana", "Haiti", "Jamaica", 
        "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", 
        "Suriname", "Trinidad and Tobago", "Aruba", "Curaçao", "Saint Martin (French part)", 
        "Sint Maarten (Dutch part)", "Turks and Caicos Islands", "Virgin Islands, British", 
        "Virgin Islands, U.S.",'Martinique','Guadeloupe'
    ],
    "Central Asia": [
        "Armenia", "Azerbaijan", "Georgia", "Kazakhstan", "Kyrgyzstan", "Mongolia", 
        "Tajikistan", "Turkmenistan", "Uzbekistan"
    ],
    "Central Europe": [
        "Albania", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Czechia", 
        "Hungary", "Kosovo", "Montenegro", "North Macedonia", "Poland", "Romania", 
        "Serbia", "Serbia and Montenegro", "Slovakia", "Slovenia"
    ],
    "Central Latin America": [
        "Colombia", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico", 
        "Nicaragua", "Panama", "Venezuela"
    ],
    "Central Sub-Saharan Africa": [
        "Angola", "Central African Republic", "Congo", "Democratic Republic of the Congo", 
        "Equatorial Guinea", "Gabon"
    ],
    "East Asia": [
        "China", "Democratic People's Republic of Korea", "Taiwan", "Hong Kong", "Macao"
    ],
    "Eastern Europe": [
        "Belarus", "Estonia", "Latvia", "Lithuania", "Republic of Moldova", 
        "Russian Federation", "Ukraine"
    ],
    "Eastern Sub-Saharan Africa": [
        "Burundi", "Comoros", "Djibouti", "Eritrea", "Ethiopia", "Kenya", "Madagascar", 
        "Malawi", "Mozambique", "Rwanda", "Somalia", "South Sudan", "Uganda", 
        "United Republic of Tanzania", "Zambia", "Réunion"
    ],
    "High-income Asia Pacific": [
        "Brunei Darussalam", "Japan", "Republic of Korea", "Singapore", "Guam", 
        "Northern Mariana Islands"
    ],
    "High-income North America": [
        "Canada", "United States of America", "Bermuda", "Puerto Rico", "Greenland", 
        "Cayman Islands"
    ],
    "North Africa and Middle East": [
        "Afghanistan", "Algeria", "Bahrain", "Egypt", "Iran", "Iraq", "Jordan", 
        "Kuwait", "Lebanon", "Libya", "Morocco", "Oman", "Palestine", "Qatar", 
        "Saudi Arabia", "Sudan", "Syrian Arab Republic", "Tunisia", "Türkiye", 
        "United Arab Emirates", "Yemen", "Israel"
    ],
    "Oceania": [
        "Fiji", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "Niue", 
        "Palau", "Papua New Guinea", "Samoa", "Solomon Islands", "Tonga", 
        "Tuvalu", "Vanuatu", "New Caledonia", "French Polynesia"
    ],
    "Southern Latin America": [
            "Argentina", "Chile", "Uruguay"
    ],
    "Southern Sub-Saharan Africa": [
        "Botswana", "Eswatini", "Lesotho", "Namibia", "South Africa", "Zimbabwe"
    ],
    "Tropical Latin America": [
        "Brazil", "Paraguay",'French Guiana'
    ],
    "Western Europe": [
        "Andorra", "Austria", "Belgium", "Cyprus", "Denmark", "Finland", "France", 
        "Germany", "Greece", "Guernsey", "Iceland", "Ireland", "Isle of Man",
        "Italy", "Jersey", "Liechtenstein", "Luxembourg", "Malta", "Monaco", 
        "Netherlands", "Norway", "Portugal", "San Marino", "Spain", "Sweden", 
        "Switzerland", "United Kingdom", "Gibraltar", "Faroe Islands"
    ],
    "Southeast Asia":[
    'Cambodia','Indonesia',"Lao People's Democratic Republic",'Malaysia','Maldives',
    'Mauritius','Myanmar','Philippines','Seychelles','Sri Lanka','Thailand','Timor-Leste',
    'Viet Nam'
    ],
    "South Asia":[
    'Bangladesh','Bhutan','India','Nepal','Pakistan'
    ],
    "Western Sub-Saharan Africa": [
        "Benin", "Burkina Faso", "Cabo Verde", "Cameroon", "Chad", "Côte d'Ivoire", 
        "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Mauritania", 
        "Niger", "Nigeria", "Sao Tome and Principe", "Senegal", "Sierra Leone", "Togo"
    ]
}

In [5]:
data = []
for region, countries in regions.items():
    for country in countries:
        data.append({"Country": country, "Region": region})

# Convert the list into a pandas DataFrame
df_country_regions = pd.DataFrame(data)

In [6]:
surface_area_countries = pd.read_csv(os.path.join(source_data_path,'area_by_country.csv'))
surface_area_countries.drop(columns=['COUNTRY'], inplace=True)

# Select the row with the highest area for each ISO3 (duplicates for islands)
surface_area_countries = surface_area_countries.loc[
    surface_area_countries.groupby('ISO3')['area_sq_km'].idxmax()
].reset_index(drop=True)

In [7]:
milk_animals=pd.read_csv(os.path.join(source_data_path,'cattle','pop_cattle_dairy_cattle.csv'))
milk_animals=milk_animals.loc[:,['Element','Area','Value','Year','ISO3']]
milk_animals.pivot(index=['Area','Year'],columns=['Element'],values=['Value']).reset_index()
milk_animals.drop(columns=['Element'],inplace=True)
milk_animals.columns=['Area','Milk Animals','Year','ISO3']
milk_animals.rename(columns={'Value':'Milk Animals','Area':'Country'},inplace=True)

In [8]:
tot_cows=pd.read_csv(os.path.join(source_data_path,'cattle','cattle_pop_2024.csv'))
tot_cows=tot_cows.loc[:,['Area','Value','Year','ISO3']]
tot_cows.rename(columns={'Value':'Total cattle','Area':'Country'},inplace=True)

In [9]:
egg_animals=pd.read_csv(os.path.join(source_data_path,'poultry','pop_egg_layers.csv'))
egg_animals=egg_animals.loc[:,['Element','Area','Value','Year','ISO3']]
egg_animals.groupby(['Area','Year']).sum().reset_index()
egg_animals['Value']*=1000
egg_animals.rename(columns={'Value':'Laying Animals','Area':'Country'},inplace=True)
egg_animals.drop(columns=['Element'],inplace=True)

In [10]:
tot_chickens=pd.read_csv(os.path.join(source_data_path,'poultry','poultry_pop_2024.csv'))
tot_chickens=tot_chickens.loc[:,['Area','Value','Year','Item','ISO3']]
tot_chickens=tot_chickens[tot_chickens['Item']=='Chickens']
tot_chickens['Value']*=1000
tot_chickens.drop(columns=['Item'],inplace=True)
tot_chickens.rename(columns={'Value':'Total chickens','Area':'Country'},inplace=True)

In [11]:
tot_poultry=pd.read_csv(os.path.join(source_data_path,'poultry','poultry_pop_2024.csv'))
tot_poultry = tot_poultry.sort_values('Value').drop_duplicates(subset=['ISO3','Year','Item'], keep='last')
tot_poultry = (
    tot_poultry.groupby(['ISO3', 'Year Code'], as_index=False)
    .agg({
        'Domain Code': 'first',
        'Domain': 'first',
        'Area Code (M49)': 'first',
        'Area': 'first',
        'Element Code': 'first',
        'Element': 'first',
        'Item Code (CPC)': 'first',
        'Year Code': 'first',
        'Year': 'first',
        'Unit': 'first',
        'Value': 'sum',  # Aggregate the 'Value' by summing
        'Flag': 'first',
        'Flag Description': 'first',
        'Note': 'first',
        'ISO3':'first'
    })
)
# Set the 'Item' column to 'Poultry'
tot_poultry=tot_poultry.sort_values('Year').loc[:,['Area','Value','ISO3','Year']]
tot_poultry['Value']*=1000
tot_poultry.rename(columns={'Value':'Total poultry','Area':'Country'},inplace=True)

tot_pigs=pd.read_csv(os.path.join(source_data_path,'swine','swine_pop_2024.csv'))
tot_pigs=tot_pigs.loc[:,['Area','Value','Year','Item','ISO3']]
tot_pigs.drop(columns=['Item'],inplace=True)
tot_pigs.rename(columns={'Value':'Total pigs','Area':'Country'},inplace=True)

In [12]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    countries['USA']='USA'
    countries['UK']='GBR'
    countries['Taiwan']='TWN'
    countries['South Korea']='KOR'
    countries['Czech Republic']='CZE'
    countries['Brunei']='BRN'
    countries['Russia']='RUS'
    countries['Iran']='IRN'
    countries['United States of America']='USA'
    countries['Venezuela']='VEN'
    countries['China (Hong Kong SAR)']='HKG'
    countries["Cote d'Ivoire"]='CIV'
    countries['DR Congo']='COD'
    countries['Guinea Bissau']='GNB'
    countries['Lao PDR']='LAO'
    countries['Micronesia (Federated States of)']='FSM'
    countries['North Korea']='PRK'
    countries['Occupied Palestinian Territory']='PSE'
    countries['Swaziland']='SWZ'
    countries['Tanzania']='TZA'
    countries['Bolivia']='BOL'
    countries['Macedonia (TFYR)']='MKD'
    countries['Moldova']='MDA'
    countries['Bolivia (Plurinational State of)']='BOL'
    countries['China, Hong Kong SAR']='HKG'
    countries['China, Taiwan Province of']='TWN'
    countries['China, mainland']='CHN'
    countries['Czechoslovakia']='CSK'
    countries["Democratic People's Republic of Korea"]='PRK'
    countries['Democratic Republic of the Congo']='COD'
    countries['French Guyana']='GUF'
    countries['Micronesia']='FSM'
    countries['Palestine']='PSE'
    countries['Polynesia']='PYF'
    countries['Republic of Korea']='KOR'
    countries['Serbia and Montenegro']='SCG'
    countries['Sudan (former)']='SDN'
    countries['Türkiye']='TUR'
    countries['USSR']='SUN'
    countries['Iran (Islamic Republic of)']='IRN'
    countries['Republic of Moldova']='MDA'
    countries['United Kingdom of Great Britain and Northern Ireland']='GBR'
    countries['United Republic of Tanzania']='TZA'
    countries['Venezuela (Bolivarian Republic of)']='VEN'
    countries['Yugoslav SFR']='YUG'
    countries['Ethiopia PDR']='ETH'
    countries['Central African (Rep.)']='CAF'
    countries["China (People's Rep. of)"]='CHN'
    countries['Chinese Taipei']='TWN'
    countries['Congo (Dem. Rep. of the)']='COD'
    countries['Congo (Rep. of the)']='COG'
    countries["Cote D'Ivoire"]='CIV'
    countries['Dominican (Rep.)']='DOM'
    countries["Korea (Dem People's Rep. of)"]='PRK'
    countries['Korea (Rep. of)']='KOR'
    countries['Laos']='LAO'
    countries['South Sudan (Rep. of)']='SSD'
    countries['Syria']='SYR'
    countries['St. Vincent and the Grenadines']='VCT'
    countries['Vietnam']='VNM'
    countries['Reunion']='REU'
    countries['Guadaloupe']='GLP'
    countries['China, Macao SAR']='MAC'
    countries['Netherlands (Kingdom of the)']='NLD'
    countries['Türkiye (Rep. of)']='TUR'
    countries['Belgium-Luxembourg']="BLX"
    countries['Kosovo']='XKX'


    
return_country = {v: k for k, v in countries.items()}


In [13]:
pop_milk = milk_animals['Country']
pop_tot_cow=tot_cows['Country']

codes_milk = [countries.get(country, 'Unknown code:'+country) for country in pop_milk]
codes_tot_cow = [countries.get(country, 'Unknown code:'+country) for country in pop_tot_cow]

pop_laying = egg_animals['Country']
pop_tot_chicken=tot_chickens['Country']

codes_laying = [countries.get(country, 'Unknown code:'+country) for country in pop_laying]
codes_tot_chicken = [countries.get(country, 'Unknown code:'+country) for country in pop_tot_chicken]

country_regions_iso3=df_country_regions['Country']
codes_regions=[countries.get(country, 'Unknown code:'+country) for country in country_regions_iso3]

iso3s_milk=[]

for i in pop_milk:
    try:
        iso3s_milk+=[countries[i]]
    except:
        iso3s_milk+=[None]
        
        
iso3s_value_cow=[]

for i in pop_tot_cow:
    try:
        iso3s_value_cow+=[countries[i]]
    except:
        iso3s_value_cow+=[None]
        
milk_animals['ISO3']=iso3s_milk
tot_cows['ISO3']=iso3s_value_cow

iso3s_egg=[]

for i in pop_laying:
    try:
        iso3s_egg+=[countries[i]]
    except:
        iso3s_egg+=[None]
        
        
iso3s_value_chicken=[]

for i in pop_tot_chicken:
    try:
        iso3s_value_chicken+=[countries[i]]
    except:
        iso3s_value_chicken+=[None]
        
        
iso3s_regions=[]

for i in country_regions_iso3:
    try:
        iso3s_regions+=[countries[i]]
    except:
        iso3s_regions+=[None]
        
        
df_country_regions['ISO3']=iso3s_regions
        
egg_animals['ISO3']=iso3s_egg
tot_chickens['ISO3']=iso3s_value_chicken
        
print(np.unique(codes_milk))

milk_animals=milk_animals.sort_values('Milk Animals').drop_duplicates(['ISO3','Year'],keep='last')
tot_cows=tot_cows.sort_values('Total cattle').drop_duplicates(['ISO3','Year'],keep='last')

egg_animals=egg_animals.sort_values('Laying Animals').drop_duplicates(['ISO3','Year'],keep='last')
tot_chickens=tot_chickens.sort_values('Total chickens').drop_duplicates(['ISO3','Year'],keep='last')

tot_pigs=tot_pigs.sort_values('Total pigs').drop_duplicates(['ISO3','Year'],keep='last')

['AFG' 'AGO' 'ALB' 'ARE' 'ARG' 'ARM' 'ATG' 'AUS' 'AUT' 'AZE' 'BDI' 'BEL'
 'BEN' 'BFA' 'BGD' 'BGR' 'BHR' 'BHS' 'BIH' 'BLR' 'BLX' 'BLZ' 'BOL' 'BRA'
 'BRB' 'BRN' 'BTN' 'BWA' 'CAF' 'CAN' 'CHE' 'CHL' 'CHN' 'CIV' 'CMR' 'COD'
 'COG' 'COL' 'COM' 'CPV' 'CRI' 'CSK' 'CUB' 'CYP' 'CZE' 'DEU' 'DJI' 'DMA'
 'DNK' 'DOM' 'DZA' 'ECU' 'EGY' 'ERI' 'ESP' 'EST' 'ETH' 'FIN' 'FJI' 'FRA'
 'FRO' 'GAB' 'GBR' 'GEO' 'GHA' 'GIN' 'GLP' 'GMB' 'GNB' 'GRC' 'GRD' 'GTM'
 'GUF' 'GUY' 'HKG' 'HND' 'HRV' 'HTI' 'HUN' 'IDN' 'IND' 'IRL' 'IRN' 'IRQ'
 'ISL' 'ISR' 'ITA' 'JAM' 'JOR' 'JPN' 'KAZ' 'KEN' 'KGZ' 'KHM' 'KOR' 'KWT'
 'LAO' 'LBN' 'LBR' 'LBY' 'LCA' 'LKA' 'LSO' 'LTU' 'LUX' 'LVA' 'MAR' 'MDA'
 'MDG' 'MEX' 'MKD' 'MLI' 'MLT' 'MMR' 'MNE' 'MNG' 'MOZ' 'MRT' 'MTQ' 'MUS'
 'MWI' 'MYS' 'NAM' 'NCL' 'NER' 'NGA' 'NIC' 'NIU' 'NLD' 'NOR' 'NPL' 'NZL'
 'OMN' 'PAK' 'PAN' 'PER' 'PHL' 'PNG' 'POL' 'PRI' 'PRK' 'PRT' 'PRY' 'PSE'
 'PYF' 'QAT' 'REU' 'ROU' 'RUS' 'RWA' 'SAU' 'SCG' 'SDN' 'SEN' 'SLB' 'SLE'
 'SLV' 'SOM' 'SRB' 'SSD' 'STP' 'SUN' 'SUR' 'SVK' 'S

In [14]:
milk_animals=pd.merge(milk_animals.drop(columns=['Country']),tot_cows.drop(columns=['Country']),how='left',on=['ISO3','Year'])
milk_animals['prop_milk_cows']=milk_animals['Milk Animals']/milk_animals['Total cattle']

In [15]:
egg_animals=pd.merge(egg_animals.drop(columns=['Country']),tot_chickens.drop(columns=['Country']),how='left',on=['ISO3','Year'])
egg_animals['prop_laying_chickens']=egg_animals['Laying Animals']/egg_animals['Total chickens']

In [16]:
data_interest=egg_animals.drop(columns=['ISO3'])

# Check for inf and -inf values in the DataFrame (X_train)
mask = np.isinf(data_interest)

# Show only rows and columns where inf/-inf values are located
inf_locations = data_interest[mask]

# Output the locations of the inf/-inf values (row, column)
rows, cols = np.where(mask)

# Print the row and column indices of the inf/-inf values
print("inf or -inf values are located at these (row, column) positions:")
for row, col in zip(rows, cols):
    print(f"Row {row}, Column {data_interest.columns[col]}")

inf or -inf values are located at these (row, column) positions:


In [17]:
#Adjusting a few cases where the number of egg-laying animals is wrong If they exist (if exist, is very small number, less than 10, maybe less than 5):
egg_animals.replace([np.inf, -np.inf], np.nan, inplace=True)

In [18]:
gdp=pd.read_csv(os.path.join(source_data_path,'GDP_per_capita.csv')).loc[:,['Country Code']+[str(i) for i in range(1960,2024,1)]]
gdp.rename(columns={'Country Code':'ISO3'},inplace=True)
gdp=gdp.melt(id_vars=['ISO3'],value_vars=[str(i) for i in range(1960,2024,1)])
gdp.rename(columns={'variable':'Year','value':'GDP per Capita'},inplace=True)
gdp=gdp[gdp['Year'].isin([str(i) for i in range(2005,2025,1)])]
gdp['Year']=gdp['Year'].astype(float)
#gdp=gdp[gdp['Year']==2023].drop(columns=['Year'])


In [19]:
tot_pigs['total pigs']=tot_pigs['Total pigs']
tot_poultry['total poultry']=tot_poultry['Total poultry']
tot_cows['total cattle']=tot_cows['Total cattle']

In [20]:
cattle_vaccine=pd.read_csv(os.path.join(notebook_dir,'Vaccination Coverage','2005-2024_full_cattle_vaccine_coverage_by_country.csv'))
cattle_vaccine.loc[cattle_vaccine['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
cattle_vaccine=cattle_vaccine.loc[:,['ISO3','Country','Year','Disease','Vaccine Coverage']]
#cattle=cattle.loc[:,['ISO3','Year','Disease','Vaccine Coverage']]
cattle_vaccine=cattle_vaccine.pivot(index=['ISO3','Country','Year'],columns=['Disease'],values=['Vaccine Coverage'])
cattle_vaccine.columns=[i[1]+'_cattle_vaccine_cov' for i in cattle_vaccine.columns]

swine_vaccine=pd.read_csv(os.path.join(notebook_dir,'Vaccination Coverage','2005-2024_full_swine_vaccine_coverage_by_country.csv'))
swine_vaccine.loc[swine_vaccine['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
swine_vaccine=swine_vaccine.loc[:,['ISO3','Disease','Year','Vaccine Coverage']]
swine_vaccine=swine_vaccine.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Vaccine Coverage'])
swine_vaccine.columns=[i[1]+'_swine_vaccine_cov' for i in swine_vaccine.columns]



poultry_vaccine=pd.read_csv(os.path.join(notebook_dir,'Vaccination Coverage','2005-2024_full_poultry_vaccine_coverage_by_country.csv'))
poultry_vaccine.loc[poultry_vaccine['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
poultry_vaccine=poultry_vaccine.loc[:,['ISO3','Year','Disease','Vaccine Coverage']]

poultry_vaccine=poultry_vaccine.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Vaccine Coverage'])
poultry_vaccine.columns=[i[1]+'_poultry_vaccine_cov' for i in poultry_vaccine.columns]


In [21]:
cattle_incidence=pd.read_csv(os.path.join(notebook_dir,'Disease Incidence','2005-2024_cattle_disease_incidence_stats.csv'))
cattle_incidence=cattle_incidence[cattle_incidence['Incidence Lower'].notna()]#Remove datapoints that could not have confidence interval generated
cattle_incidence=cattle_incidence.loc[:,['ISO3','Country','Year','Disease','Incidence']]
# Will be reporting cases per 100,000; incidence is currently = cases per animal
cattle_incidence['Incidence']*=100000
cattle_incidence=cattle_incidence.pivot(index=['ISO3','Country','Year'],columns=['Disease'],values=['Incidence'])
cattle_incidence.columns=[i[1]+'_cattle_incidence' for i in cattle_incidence.columns]


swine_incidence=pd.read_csv(os.path.join(notebook_dir,'Disease Incidence','2005-2024_swine_disease_incidence_stats.csv'))
swine_incidence=swine_incidence[swine_incidence['Incidence Lower'].notna()]#Remove datapoints that could not have confidence interval generated
swine_incidence=swine_incidence.loc[:,['ISO3','Disease','Year','Incidence']]
# Will be reporting cases per 100,000; incidence is currently = cases per animal
swine_incidence['Incidence']*=100000
swine_incidence=swine_incidence.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Incidence'])
swine_incidence.columns=[i[1]+'_swine_incidence' for i in swine_incidence.columns]
poultry_incidence=pd.read_csv(os.path.join(notebook_dir,'Disease Incidence','2005-2024_poultry_disease_incidence_stats.csv'))
poultry_incidence=poultry_incidence[poultry_incidence['Incidence Lower'].notna()]#Remove datapoints that could not have confidence interval generated
poultry_incidence=poultry_incidence.loc[:,['ISO3','Disease','Year','Incidence']]
# Will be reporting cases per 100,000; incidence is currently = cases per animal
poultry_incidence['Incidence']*=100000
poultry_incidence=poultry_incidence.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Incidence'])
poultry_incidence.columns=[i[1]+'_poultry_incidence' for i in poultry_incidence.columns]


df_impute_original=reduce(lambda  left,right: pd.merge(left,right,on=['ISO3','Year'],
                                            how='outer'), [cattle_incidence,swine_incidence,poultry_incidence,
                                                           cattle_vaccine,swine_vaccine,poultry_vaccine,
                                                           gdp,milk_animals,egg_animals,
                                                          tot_pigs.drop(columns=['Country']),
                                                           tot_poultry.drop(columns=['Country','Total poultry']),
                                                           tot_cows.drop(columns=['Country','Total cattle'])])

df_impute_original=df_impute_original.merge(surface_area_countries,how='left',on=['ISO3'])

df_impute_original=pd.merge(df_impute_original,df_country_regions,on='ISO3',how='left')
df_impute_original=df_impute_original.drop(columns=['Milk Animals','Total cattle', 'Laying Animals','Total chickens', 'Total pigs'])

df_impute_original=df_impute_original[(df_impute_original['Year']>=2005)&(df_impute_original['Year']<=2024)]

drop_these_iso3s=np.unique(df_impute_original[df_impute_original['Country'].isna()]['ISO3'])
df_impute_original = df_impute_original[~df_impute_original['ISO3'].isin(drop_these_iso3s)]

df_impute_original['pig_density']=df_impute_original['total pigs']/df_impute_original['area_sq_km']
df_impute_original['poultry_density']=df_impute_original['total poultry']/df_impute_original['area_sq_km']
df_impute_original['cattle_density']=df_impute_original['total cattle']/df_impute_original['area_sq_km']

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the 'Region' column to integers
df_impute_original['Region_encoded'] = le.fit_transform(df_impute_original['Region'])
df_impute_original.drop(columns=['Region','Country'],inplace=True)

In [22]:
cattle_vaccine=pd.read_csv(os.path.join(notebook_dir,'Vaccination Coverage','2005-2024_full_cattle_vaccine_coverage_by_country.csv'))
cattle_vaccine.loc[cattle_vaccine['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
cattle_vaccine=cattle_vaccine.loc[:,['ISO3','Country','Year','Disease','Vaccine Coverage','Vaccine Coverage Lower','Vaccine Coverage Upper']]
cattle_vaccine.drop_duplicates(inplace=True)
cattle_vaccine=cattle_vaccine.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Vaccine Coverage','Vaccine Coverage Lower','Vaccine Coverage Upper'])
#Rename columns to include the naming convention
cattle_vaccine.columns = [f"{disease}_cattle_vaccine_cov" if var == 'Vaccine Coverage' else 
                   f"{disease}_cattle_vaccine_cov_lower" if var == 'Vaccine Coverage Lower' else 
                   f"{disease}_cattle_vaccine_cov_upper" 
                   for var, disease in cattle_vaccine.columns]

swine_vaccine=pd.read_csv(os.path.join(notebook_dir,'Vaccination Coverage','2005-2024_full_swine_vaccine_coverage_by_country.csv'))
swine_vaccine.loc[swine_vaccine['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
swine_vaccine=swine_vaccine.loc[:,['ISO3','Disease','Year','Vaccine Coverage','Vaccine Coverage Lower','Vaccine Coverage Upper']]
swine_vaccine.drop_duplicates(inplace=True)
swine_vaccine=swine_vaccine.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Vaccine Coverage','Vaccine Coverage Lower','Vaccine Coverage Upper'])
#Rename columns to include the naming convention
swine_vaccine.columns = [f"{disease}_swine_vaccine_cov" if var == 'Vaccine Coverage' else 
                   f"{disease}_swine_vaccine_cov_lower" if var == 'Vaccine Coverage Lower' else 
                   f"{disease}_swine_vaccine_cov_upper" 
                   for var, disease in swine_vaccine.columns]


poultry_vaccine=pd.read_csv(os.path.join(notebook_dir,'Vaccination Coverage','2005-2024_full_poultry_vaccine_coverage_by_country.csv'))
poultry_vaccine.loc[poultry_vaccine['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
poultry_vaccine = poultry_vaccine.loc[:, ['ISO3', 'Year', 'Disease', 'Vaccine Coverage', 'Vaccine Coverage Lower', 'Vaccine Coverage Upper']]
poultry_vaccine.drop_duplicates(inplace=True)

# Pivot table with new column naming convention
poultry_vaccine = poultry_vaccine.pivot(index=['ISO3', 'Year'], columns=['Disease'], values=['Vaccine Coverage', 'Vaccine Coverage Lower', 'Vaccine Coverage Upper'])

#Rename columns to include the naming convention
poultry_vaccine.columns = [f"{disease}_poultry_vaccine_cov" if var == 'Vaccine Coverage' else 
                   f"{disease}_poultry_vaccine_cov_lower" if var == 'Vaccine Coverage Lower' else 
                   f"{disease}_poultry_vaccine_cov_upper" 
                   for var, disease in poultry_vaccine.columns]

poultry_vaccine.reset_index(inplace=True)

In [23]:
cattle_incidence=pd.read_csv(os.path.join(notebook_dir,'Disease Incidence','2005-2024_cattle_disease_incidence_stats.csv'))
cattle_incidence=cattle_incidence.loc[:,['ISO3','Country','Year','Disease','Incidence','Incidence Lower','Incidence Upper']]
# Will be reporting cases per 100,000; incidence is currently = cases per animal
cattle_incidence['Incidence']*=100000
cattle_incidence['Incidence Lower']*=100000
cattle_incidence['Incidence Upper']*=100000
cattle_incidence=cattle_incidence[cattle_incidence['Incidence Lower'].notna()]#Remove datapoints that could not have confidence interval generated
cattle_incidence=cattle_incidence.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Incidence','Incidence Lower','Incidence Upper'])
# Rename columns to include the naming convention
cattle_incidence.columns = [f"{disease}_cattle_incidence" if var == 'Incidence' else 
                   f"{disease}_cattle_incidence_lower" if var == 'Incidence Lower' else 
                   f"{disease}_cattle_incidence_upper" 
                   for var, disease in cattle_incidence.columns]


swine_incidence=pd.read_csv(os.path.join(notebook_dir,'Disease Incidence','2005-2024_swine_disease_incidence_stats.csv'))
swine_incidence=swine_incidence.loc[:,['ISO3','Disease','Year','Incidence','Incidence Lower','Incidence Upper']]
# Will be reporting cases per 100,000; incidence is currently = cases per animal
swine_incidence['Incidence']*=100000
swine_incidence['Incidence Lower']*=100000
swine_incidence['Incidence Upper']*=100000
swine_incidence=swine_incidence[swine_incidence['Incidence Lower'].notna()]#Remove datapoints that could not have confidence interval generated
swine_incidence=swine_incidence.pivot(index=['ISO3','Year'],columns=['Disease'],values=['Incidence','Incidence Lower','Incidence Upper'])
# Rename columns to include the naming convention
swine_incidence.columns = [f"{disease}_swine_incidence" if var == 'Incidence' else 
                   f"{disease}_swine_incidence_lower" if var == 'Incidence Lower' else 
                   f"{disease}_swine_incidence_upper" 
                   for var, disease in swine_incidence.columns]

poultry_incidence=pd.read_csv(os.path.join(notebook_dir,'Disease Incidence','2005-2024_poultry_disease_incidence_stats.csv'))
poultry_incidence.loc[poultry_incidence['Country'] == 'Türkiye (Rep. of)', 'ISO3'] = 'TUR'
poultry_incidence = poultry_incidence.loc[:, ['ISO3', 'Year', 'Disease', 'Incidence', 'Incidence Lower', 'Incidence Upper']]

# Will be reporting cases per 100,000; incidence is currently = cases per animal
poultry_incidence['Incidence']*=100000
poultry_incidence['Incidence Lower']*=100000
poultry_incidence['Incidence Upper']*=100000
poultry_incidence=poultry_incidence[poultry_incidence['Incidence Lower'].notna()]#Remove datapoints that could not have confidence interval generated
# Pivot table with new column naming convention
poultry_incidence = poultry_incidence.pivot(index=['ISO3', 'Year'], columns=['Disease'], values=['Incidence', 'Incidence Lower', 'Incidence Upper'])

# Rename columns to include the naming convention
poultry_incidence.columns = [f"{disease}_poultry_incidence" if var == 'Incidence' else 
                   f"{disease}_poultry_incidence_lower" if var == 'Incidence Lower' else 
                   f"{disease}_poultry_incidence_upper" 
                   for var, disease in poultry_incidence.columns]

poultry_incidence.reset_index(inplace=True)


df_impute_sampling=reduce(lambda  left,right: pd.merge(left,right,on=['ISO3','Year'],
                                            how='outer'), [cattle_incidence,swine_incidence,poultry_incidence,
                                                           cattle_vaccine,swine_vaccine,poultry_vaccine,
                                                           gdp,milk_animals,egg_animals,
                                                          tot_pigs.drop(columns=['Country']),
                                                           tot_poultry.drop(columns=['Country','Total poultry']),
                                                           tot_cows.drop(columns=['Country','Total cattle'])])

df_impute_sampling=df_impute_sampling.merge(surface_area_countries,how='left',on=['ISO3'])

df_impute_sampling=df_impute_sampling.drop(columns=['Milk Animals','Total cattle', 'Laying Animals','Total chickens','Total pigs'])

df_impute_sampling=df_impute_sampling[(df_impute_sampling['Year']>=2005)&(df_impute_sampling['Year']<=2024)]
df_impute_sampling = df_impute_sampling[~df_impute_sampling['ISO3'].isin(drop_these_iso3s)]
df_impute_sampling=pd.merge(df_impute_sampling,df_country_regions,on='ISO3',how='left')

df_impute_sampling['pig_density']=df_impute_sampling['total pigs']/df_impute_sampling['area_sq_km']
df_impute_sampling['poultry_density']=df_impute_sampling['total poultry']/df_impute_sampling['area_sq_km']
df_impute_sampling['cattle_density']=df_impute_sampling['total cattle']/df_impute_sampling['area_sq_km']

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the 'Region' column to integers
df_impute_sampling['Region_encoded'] = le.fit_transform(df_impute_sampling['Region'])
df_impute_sampling.drop(columns=['Region'],inplace=True)

In [24]:
total_rows_removed = 0
total_rows_processed = 0

for col in df_impute_original.columns:
    if ('_swine_incidence' in col ) or ('_cattle_incidence' in col) or ('_poultry_incidence' in col):
        
        if '_swine' in col:
            pop_size_col='total pigs'
        elif '_cattle' in col:
            pop_size_col='total cattle'
        else:
            pop_size_col='total poultry'


        disease_col = col
        data = df_impute_original.copy()
        iso3_col = 'ISO3'
        min_pop_threshold = 50000

        max_incidence_per_country = data.groupby(iso3_col)[disease_col].max()

        epsilon = 1e-9 # Small constant to avoid log(0)
        max_incidence_per_country_log = np.log10(max_incidence_per_country + epsilon)

        #Computing thresholds for high outliers using Modified Z-Score
        median_log_max_incidence = max_incidence_per_country_log.median()
        mad_log_max_incidence = np.median(np.abs(
            np.array([i for i in max_incidence_per_country_log.values if i == i]) - median_log_max_incidence
        ))
        outlier_threshold = 3.5

        # Identifying countries with outlier maximum incidence values
        outlier_countries = max_incidence_per_country_log[
            0.6745 * (max_incidence_per_country_log - median_log_max_incidence) / mad_log_max_incidence > outlier_threshold
        ].index

        data["Too_Small_Pop"] = data[pop_size_col] < min_pop_threshold

        data["Is_Outlier_Country"] = data[iso3_col].isin(outlier_countries)
        filtered_data = data[~(data["Is_Outlier_Country"] & data["Too_Small_Pop"])]

        rows_removed = len(data) - len(filtered_data)
        total_rows_removed += rows_removed
        total_rows_processed += len(data)
        
        if len(np.unique(data[data["Is_Outlier_Country"]&data["Too_Small_Pop"]]['ISO3']))>0:

            print("Unique ISO3 values of outlier countries:")
            print(np.unique(data[data["Is_Outlier_Country"]&data["Too_Small_Pop"]]['ISO3']))

            max_values_by_iso3 = data[data["Is_Outlier_Country"]&data["Too_Small_Pop"]].groupby('ISO3')[disease_col].max()
            print("Max values by ISO3:")
            print(max_values_by_iso3)

        filtered_data = filtered_data.drop(columns=["Too_Small_Pop", "Is_Outlier_Country"], errors="ignore")

proportion_removed = total_rows_removed / total_rows_processed
print(f"Percent of rows removed: {proportion_removed*100:.6f}%")

Unique ISO3 values of outlier countries:
['BWA' 'TUR']
Max values by ISO3:
ISO3
BWA    365.038723
TUR    105.485232
Name: Rabies virus (Inf. with)_swine_incidence, dtype: float64
Percent of rows removed: 0.005896%


In [25]:
def identify_outliers(data, disease_col, pop_size_col, iso3_col='ISO3', year_col='Year',
                      min_pop_threshold=50000, outlier_threshold=3.5):#min_pop_threshold=120000, outlier_threshold=2.5):
    """
    Identifies ISO3, Year pairs that meet the outlier criteria based on maximum disease incidence
    and population size thresholds.

    Parameters:
        data (pd.DataFrame): The input dataframe.
        disease_col (str): The name of the disease incidence column.
        pop_size_col (str): The name of the animal population size column.
        iso3_col (str): The name of the ISO3 country code column.
        year_col (str): The name of the year column.
        min_pop_threshold (int, optional): The minimum population size threshold (default=50,000.
        outlier_threshold (float, optional): The Modified Z-Score threshold for outliers (default=3.5)

    Returns:
        set: A set of tuples representing the identified (ISO3, Year) pairs.
    """
    data = data.copy()  

    #Calculate the maximum incidence per country (ISO3)
    max_incidence_per_country = data.groupby(iso3_col)[disease_col].max()

    # Log-transform the maximum incidence values
    epsilon = 1e-9  # Small constant to avoid log(0)
    max_incidence_per_country_log = np.log10(max_incidence_per_country + epsilon)

    # ompute thresholds for high outliers using Modified Z-Score
    median_log_max_incidence = max_incidence_per_country_log.median()
    mad_log_max_incidence = np.median(
        np.abs(np.array([i for i in max_incidence_per_country_log.values if i == i]) - median_log_max_incidence)
    )

    outlier_countries = max_incidence_per_country_log[
        0.6745 * (max_incidence_per_country_log - median_log_max_incidence) / mad_log_max_incidence > outlier_threshold
    ].index

    # Apply the fixed minimum population size threshold
    data["Too_Small_Pop"] = data[pop_size_col] < min_pop_threshold
    data["Is_Outlier_Country"] = data[iso3_col].isin(outlier_countries)

    # Identify rows to flag based on actual values in 'disease_col'
    rows_to_flag = (
        data["Is_Outlier_Country"] & data["Too_Small_Pop"] & data[disease_col].notna()
    )

    flagged_iso3_year_pairs = set(
        data.loc[rows_to_flag, [iso3_col, year_col]].drop_duplicates().itertuples(index=False, name=None)
    )

    return flagged_iso3_year_pairs


# XGBoost

In [26]:
# Clip function to bound predictions between 0 and 1
def clip_values(y_pred,col_name):
    if 'incidence' in col_name:
        return np.maximum(y_pred, 0)
    else:
        return np.clip(y_pred, 0, 1)

In [27]:
columns_interest=[i for i in df_impute_original.columns if i not in ['ISO3','Year','GDP per Capita','prop_milk_cows','prop_laying_chickens',
                                                   'Region_encoded']]

In [28]:
len(columns_interest)

308

In [29]:
try:
    # Clear the optuna.log file
    with open("optuna.log", "w") as f:
        pass  # Opening in 'w' mode clears the file
except:
    pass

# Get Optuna logger
optuna_logger = logging.getLogger("optuna")

optuna_logger.setLevel(logging.INFO)

# Remove default console (stream) handler to prevent output in the notebook
optuna_logger.handlers = []

# Add a file handler to redirect logs to a file
file_handler = logging.FileHandler("optuna.log")
optuna_logger.addHandler(file_handler)


In [30]:
df_pop_cattle=pd.read_csv(os.path.join(source_data_path, 'cattle','cattle_pop_2024.csv'))
df_pop_cattle.rename(columns={'Value':'latest pop cattle'},inplace=True)
df_pop_cattle=df_pop_cattle.sort_values('Year').drop_duplicates(['ISO3','Year'],keep='last').loc[:,['ISO3','latest pop cattle','Year']]
df_pop_cattle['Animal']=['Cattle']*df_pop_cattle.shape[0]

df_pop_poultry=pd.read_csv(os.path.join(source_data_path, 'poultry','poultry_pop_2024.csv'))
df_pop_poultry = (
    df_pop_poultry.groupby(['Area Code (M49)', 'Year Code'], as_index=False)
    .agg({
        'Domain Code': 'first',
        'ISO3':'first',
        'Domain': 'first',
        'Area Code (M49)': 'first',
        'Area': 'first',
        'Element Code': 'first',
        'Element': 'first',
        'Item Code (CPC)': 'first',
        'Year Code': 'first',
        'Year': 'first',
        'Unit': 'first',
        'Value': 'sum',  
        'Flag': 'first',
        'Flag Description': 'first',
        'Note': 'first'
    })
)
df_pop_poultry['Item'] = 'Poultry'
df_pop_poultry=df_pop_poultry.sort_values('Year').drop_duplicates(['ISO3','Year'],keep='last').loc[:,['ISO3','Value','Year']]
df_pop_poultry.rename(columns={'Value':'latest pop poultry'},inplace=True)
df_pop_poultry['Animal']=['Poultry']*df_pop_poultry.shape[0]
df_pop_poultry['latest pop poultry']*=1000


df_pop_swine=pd.read_csv(os.path.join(source_data_path, 'swine','swine_pop_2024.csv'))
df_pop_swine.rename(columns={'Value':'latest pop swine'},inplace=True)
df_pop_swine['Animal']=['Pigs']*df_pop_swine.shape[0]



In [31]:
df_killed_pop_cattle=pd.read_csv(os.path.join(source_data_path, 'cattle','killed_cattle_pop_2024.csv'))
df_killed_pop_cattle.rename(columns={'Value':'killed pop cattle'},inplace=True)
df_killed_pop_cattle=df_killed_pop_cattle.sort_values('Year').drop_duplicates(['ISO3','Year'],keep='last').loc[:,['ISO3','killed pop cattle','Year']]
df_killed_pop_cattle['Animal']=['Cattle']*df_killed_pop_cattle.shape[0]


df_killed_pop_poultry=pd.read_csv(os.path.join(source_data_path, 'poultry','killed_poultry_pop_2024.csv'))
df_killed_pop_poultry = (
    df_killed_pop_poultry.groupby(['Area Code (M49)', 'Year Code'], as_index=False)
    .agg({
        'Domain Code': 'first',
        'Domain': 'first',
        'Area Code (M49)': 'first',
        'ISO3':'first',
        'Area': 'first',
        'Element Code': 'first',
        'Element': 'first',
        'Item Code (CPC)': 'first',
        'Year Code': 'first',
        'Year': 'first',
        'Unit': 'first',
        'Value': 'sum',  
        'Flag': 'first',
        'Flag Description': 'first',
        'Note': 'first'
    })
)
df_killed_pop_poultry['Item'] = 'Poultry'
df_killed_pop_poultry=df_killed_pop_poultry.sort_values('Year').drop_duplicates(['ISO3','Year'],keep='last').loc[:,['ISO3','Value','Year']]
df_killed_pop_poultry.rename(columns={'Value':'killed pop poultry'},inplace=True)
df_killed_pop_poultry['Animal']=['Poultry']*df_killed_pop_poultry.shape[0]
df_killed_pop_poultry['killed pop poultry']*=1000


df_killed_pop_swine=pd.read_csv(os.path.join(source_data_path, 'swine','killed_swine_pop_2024.csv')).sort_values('Year').drop_duplicates(['ISO3','Year'],keep='last').loc[:,['ISO3','Value','Year']]
df_killed_pop_swine.rename(columns={'Value':'killed pop swine'},inplace=True)
df_killed_pop_swine['Animal']=['Pigs']*df_killed_pop_swine.shape[0]


In [32]:
df_impute_original=df_impute_original.merge(df_pop_cattle.drop(columns=['Animal']),how='left',on=['ISO3','Year'])
df_impute_original=df_impute_original.merge(df_pop_swine.drop(columns=['Animal']),how='left',on=['ISO3','Year'])
df_impute_original=df_impute_original.merge(df_pop_poultry.drop(columns=['Animal']),how='left',on=['ISO3','Year'])

df_impute_original=df_impute_original.merge(df_killed_pop_cattle.drop(columns=['Animal']),how='left',on=['ISO3','Year'])
df_impute_original=df_impute_original.merge(df_killed_pop_swine.drop(columns=['Animal']),how='left',on=['ISO3','Year'])
df_impute_original=df_impute_original.merge(df_killed_pop_poultry.drop(columns=['Animal']),how='left',on=['ISO3','Year'])


df_impute_original['total_cattle']=df_impute_original['latest pop cattle']+df_impute_original['killed pop cattle']
df_impute_original['total_poultry']=df_impute_original['latest pop poultry']+df_impute_original['killed pop poultry']
df_impute_original['total_swine']=df_impute_original['latest pop swine']+df_impute_original['killed pop swine']

In [33]:
total_cattle_weights=np.nan_to_num(df_impute_original['total_cattle'], nan=0)
total_poultry_weights=np.nan_to_num(df_impute_original['total_poultry'], nan=0)
total_swine_weights=np.nan_to_num(df_impute_original['total_swine'], nan=0)

In [34]:
# Get the index of the column "Region_encoded"
col_index = df_impute_original.columns.get_loc("Region_encoded")

# Keep only columns up to "Region_encoded" (inclusive)
df_impute_original = df_impute_original.iloc[:, :col_index + 1]

In [35]:
def objective(trial, X_train, y_train, countries, feature_importance, mandatory_features):
    # Map each row to its corresponding country
    unique_countries = pd.Series(countries).unique()

    # Define Group K-Fold with 5 random splits
    np.random.seed(15)  # Ensure reproducibility
    k = 5
    random_country_splits = np.array_split(np.random.permutation(unique_countries), k)

    # Suggesting hyperparameters for XGBoost
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'random_state': 22,
        'enable_categorical': True,
        'eval_metric': 'mae',
        'early_stopping_rounds': 10,
        'tree_method': 'hist'
    }

    # Suggesting the percentage of features to keep
    feature_space_percentage = trial.suggest_float('feature_percentage', 0.025, 0.125)  # 2.5% to 12.5%

    # Calculates the number of features to keep
    num_top_features = int(len(feature_importance) * feature_space_percentage)

    mandatory_indices = [i for i, col in enumerate(X_train.columns) if col in mandatory_features]

    non_mandatory_indices = [
        i for i in range(len(feature_importance)) if i not in mandatory_indices
    ]
    sorted_non_mandatory_indices = [non_mandatory_indices[i] for i in np.argsort(feature_importance[non_mandatory_indices])[::-1]]
    top_disease_feature_indices = sorted_non_mandatory_indices[:num_top_features]

    # Combine mandatory and selected non-mandatory features
    final_feature_indices = set(top_disease_feature_indices).union(mandatory_indices)
    top_features = [X_train.columns[i] for i in final_feature_indices]

    X_train_selected = X_train[top_features]

    mae_scores = []

    for validation_split in random_country_splits:
        # Create train and validation splits based on countries
        is_valid = pd.Series(countries).isin(validation_split).values
        is_train = ~is_valid

        X_tr, X_val = X_train_selected.iloc[is_train], X_train_selected.iloc[is_valid]
        y_tr, y_val = y_train.iloc[is_train], y_train.iloc[is_valid]

        # Train the model
        model = XGBRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        preds = model.predict(X_val)
        mae_scores.append(mean_absolute_error(y_val, preds))

    # Return average MAE across splits
    return sum(mae_scores) / len(mae_scores)


In [36]:
def compute_feature_importance_group_cv(X_train, y_train, countries, model, kf_splits=10):
    # Unique countries and random splits for group CV
    unique_countries = pd.Series(countries).unique()
    np.random.seed(42)
    random_country_splits = np.array_split(np.random.permutation(unique_countries), kf_splits)
    
    feature_importance = np.zeros(len(X_train.columns))
    
    for validation_split in random_country_splits:
        # Create train and validation splits based on countries
        is_valid = pd.Series(countries).isin(validation_split).values
        is_train = ~is_valid

        X_tr, X_val = X_train.iloc[is_train], X_train.iloc[is_valid]
        y_tr, y_val = y_train.iloc[is_train], y_train.iloc[is_valid]

        # Fit the model and get feature importances
        model.fit(X_tr, y_tr,)
        feature_importance += model.feature_importances_ / kf_splits

    return feature_importance


In [37]:
len(df_impute_original.iloc[:,:-11].columns)-2

301

In [38]:
df_impute_original.iloc[:,:-11].columns

Index(['ISO3', 'Year',
       'African swine fever virus (Inf. with)_cattle_incidence',
       'Anthrax_cattle_incidence',
       'Aujeszky's disease virus (Inf. with)_cattle_incidence',
       'Avian chlamydiosis_cattle_incidence',
       'Bluetongue virus (Inf. with)_cattle_incidence',
       'Bovine anaplasmosis_cattle_incidence',
       'Bovine babesiosis_cattle_incidence',
       'Bovine cysticercosis (-2014)_cattle_incidence',
       ...
       'Pigeon rotavirus_poultry_vaccine_cov',
       'Pullorum disease_poultry_vaccine_cov',
       'Rabies virus (Inf. with)_poultry_vaccine_cov',
       'Reovirus (inf with)_poultry_vaccine_cov',
       'Salmonellosis (S. abortusovis)_poultry_vaccine_cov',
       'Sheep pox and goat pox_poultry_vaccine_cov',
       'Trichomonosis_poultry_vaccine_cov',
       'Trypanosomosis (tsetse-transmitted) (-2021)_poultry_vaccine_cov',
       'Turkey rhinotracheitis (2006-)_poultry_vaccine_cov',
       'West Nile Fever_poultry_vaccine_cov'],
      dtype='

In [39]:
#Shuffling column order so that iterative imputation equally benefits imputations of disease incidence and vaccination coverage
# Identify the columns
first_two_columns = df_impute_original.columns[:2]
last_11_columns = df_impute_original.columns[-11:]
middle_columns = df_impute_original.columns[2:-11]

# Shuffle the middle columns
shuffled_middle_columns = random.sample(list(middle_columns), len(middle_columns))

# Combine the columns
new_column_order = list(first_two_columns) + shuffled_middle_columns + list(last_11_columns)

# Reorder the DataFrame
df_impute_original = df_impute_original[new_column_order]

df_impute_original.head()

,ISO3,Year,Anthrax_swine_incidence,Bovine tuberculosis (-2018)_cattle_vaccine_cov,Taenia solium (Inf. with) (Porcine cysticercosis)_swine_incidence,New world screwworm (Cochliomyia hominivorax)_cattle_incidence,Influenza A virus (Inf. with)_swine_vaccine_cov,Salmonellosis (S. abortusovis)_cattle_incidence,Rabies virus (Inf. with)_swine_incidence,Swine vesicular disease (-2014)_swine_incidence,...,prop_milk_cows,prop_laying_chickens,total pigs,total poultry,total cattle,area_sq_km,pig_density,poultry_density,cattle_density,Region_encoded
0,AFG,2005.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.698362,0.846399,NaN,14414000.0,3723000.0,930332.238549,NaN,15.493390,4.001796,12
1,AFG,2006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.681265,0.845588,NaN,10880000.0,4110000.0,930332.238549,NaN,11.694747,4.417777,12
2,AFG,2007.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.688547,0.907582,NaN,9035000.0,4357000.0,930332.238549,NaN,9.711584,4.683273,12
3,AFG,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.695469,0.851343,NaN,10689000.0,4745000.0,930332.238549,NaN,11.489444,5.100328,12
4,AFG,2009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.699004,0.853527,NaN,10193000.0,4721000.0,930332.238549,NaN,10.956301,5.074531,12


In [40]:
def sample_from_gamma_vectorized(lower, upper, mean, n_samples, rng):
    # Initialize the output array with NaN as default
    samples = np.full((len(mean) * n_samples), np.nan)

    # Handle cases where mean == 0
    mean = np.where(mean == 0, 0.001, mean)

    # Swap bounds where lower > upper
    invalid_bounds = lower > upper
    if invalid_bounds.any():
        lower[invalid_bounds], upper[invalid_bounds] = upper[invalid_bounds], lower[invalid_bounds]

    # Identify rows where both lower and upper are 0
    zero_bounds = (lower == 0) & (upper == 0)
    if zero_bounds.any():
        zero_indices = np.where(zero_bounds)[0]
        for zero_row in zero_indices:
            samples[zero_row * n_samples:(zero_row + 1) * n_samples] = 0

    #Identify valid rows where lower, upper, and mean are not NaN
    valid_rows = ~np.isnan(mean) & ~np.isnan(lower) & ~np.isnan(upper) & ~zero_bounds

    if valid_rows.any():
        # Calculate variance, shape, and scale for valid rows
        variance = ((upper[valid_rows] - lower[valid_rows]) / 4) ** 2
        variance = np.where(variance == 0, 1e-6, variance)  # Handle zero variance

        shape = mean[valid_rows] ** 2 / variance
        scale = variance / mean[valid_rows]

        # Separate valid and invalid gamma parameter rows
        valid_params = (shape > 0) & (scale > 0)

        #Handle rows with valid gamma parameters
        valid_indices = np.where(valid_rows)[0][valid_params]
        if valid_params.any():
            for i, valid_row in enumerate(valid_indices):
                sampled_values = rng.gamma(shape[valid_params][i], scale[valid_params][i], n_samples)
                samples[valid_row * n_samples:(valid_row + 1) * n_samples] = sampled_values

        # Handle rows with invalid gamma parameters
        invalid_params = ~valid_params
        invalid_indices = np.where(valid_rows)[0][invalid_params]
        for invalid_row in invalid_indices:
            if lower[invalid_row] <= upper[invalid_row]:  # Ensure valid bounds
                sampled_values = rng.uniform(lower[invalid_row], upper[invalid_row], n_samples)
                samples[invalid_row * n_samples:(invalid_row + 1) * n_samples] = sampled_values

    return samples


In [41]:
def sample_from_beta_vectorized(lower, upper, mean, n_samples, rng):
    # Initialize the output array with NaN as default
    samples = np.full((len(mean) * n_samples), np.nan)

    # Handle cases where mean == 0
    mean = np.where(mean == 0, 0.001, mean)

    # Swap bounds where lower > upper
    invalid_bounds = lower > upper
    if invalid_bounds.any():
        lower[invalid_bounds], upper[invalid_bounds] = upper[invalid_bounds], lower[invalid_bounds]

    # Identify valid rows where lower, upper, and mean are not NaN
    valid_rows = ~np.isnan(mean) & ~np.isnan(lower) & ~np.isnan(upper)

    if valid_rows.any():
        # Calculate variance, alpha, and beta for valid rows
        variance = ((upper[valid_rows] - lower[valid_rows]) / 4) ** 2
        variance = np.where(variance == 0, 1e-6, variance)  # Handle zero variance

        alpha = ((mean[valid_rows] ** 2) * (1 - mean[valid_rows]) / variance) - mean[valid_rows]
        beta = alpha * (1 / mean[valid_rows] - 1)

        # Separate valid and invalid beta parameter rows
        valid_params = (alpha > 0) & (beta > 0)

        # Handle rows with valid beta parameters
        valid_indices = np.where(valid_rows)[0][valid_params]
        if valid_params.any():
            for i, valid_row in enumerate(valid_indices):
                sampled_values = rng.beta(alpha[valid_params][i], beta[valid_params][i], n_samples)
                samples[valid_row * n_samples:(valid_row + 1) * n_samples] = sampled_values

        #Handle rows with invalid beta parameters
        invalid_params = ~valid_params
        invalid_indices = np.where(valid_rows)[0][invalid_params]
        for invalid_row in invalid_indices:
            if lower[invalid_row] <= upper[invalid_row]:  # Ensure valid bounds
                sampled_values = rng.uniform(lower[invalid_row], upper[invalid_row], n_samples)
                samples[invalid_row * n_samples:(invalid_row + 1) * n_samples] = sampled_values

    return samples


In [42]:
#This is done to remove outliers from influencing imputations 
def update_flagged_diseases(df, diseases_flagged):
    """
    Set flagged disease columns to NaN for rows with matching ISO3 and Year pairs.

    Parameters:
        df (pd.DataFrame): The dataframe to update.
        diseases_flagged (dict): A dictionary where keys are column names (diseases) 
                                 and values are sets of (ISO3, Year) pairs.

    Returns:
        None (modifies df in place)
    """
    for disease_column, flagged_pairs in diseases_flagged.items():
        if flagged_pairs:  
            # Create a mask for rows matching the flagged ISO3-Year pairs
            mask = df.apply(lambda row: (row['ISO3'], row['Year']) in flagged_pairs, axis=1)
            print(f'Identified outliers for disease: {disease_column}')
            print(f"Flagged and removed pairs (extreme outliers): {flagged_pairs}")

            # Set the disease column values to NaN for the flagged rows
            df.loc[mask, disease_column] = np.nan


In [43]:
bold_start = '\033[1m'
bold_end = '\033[0m'

In [ ]:
n_samples = 50
n_bootstrap = 100
bayesian_trials=1000 #Iterations of bayesian tuning

seed = 2
rng = np.random.default_rng(seed)



# List of columns to process
incidence_cols = [col for col in df_impute_sampling.columns if ('_incidence' in col) & ('upper' not in col) & ('lower' not in col)]
vaccine_cols = [col for col in df_impute_sampling.columns if ('_vaccine' in col) & ('upper' not in col) & ('lower' not in col)]

# Reused variables
static_columns = ['ISO3', 'Year', 'GDP per Capita', 'prop_milk_cows', 'prop_laying_chickens', 
                  'Region_encoded', 'total poultry', 'total cattle', 'total pigs', 
                  'area_sq_km', 'cattle_density', 'poultry_density', 'pig_density']

# Repeat rows of static columns to match the number of samples
static_data = df_impute_sampling[static_columns].loc[
    np.repeat(df_impute_sampling.index, n_samples)
].reset_index(drop=True)

synthetic_data_dict={}

for col in static_columns:
    synthetic_data_dict[col] = static_data[col].values

synthetic_df = pd.DataFrame(synthetic_data_dict)


sampled_columns = {}

#These are samples from the CIs of estimates
for col in incidence_cols:
    lower = df_impute_sampling[col.replace('_incidence', '_incidence_lower')].values
    upper = df_impute_sampling[col.replace('_incidence', '_incidence_upper')].values
    mean = df_impute_sampling[col].values

    sampled_values = sample_from_gamma_vectorized(lower, upper, mean, n_samples, rng)
    
    sampled_columns[col] = sampled_values

for col in vaccine_cols:
    lower = df_impute_sampling[col.replace('_vaccine_cov', '_vaccine_cov_lower')].values
    upper = df_impute_sampling[col.replace('_vaccine_cov', '_vaccine_cov_upper')].values
    mean = df_impute_sampling[col].values

    sampled_values = sample_from_beta_vectorized(lower, upper, mean, n_samples, rng)
    
    sampled_columns[col] = sampled_values

sampled_df = pd.DataFrame(sampled_columns)
synthetic_df = pd.concat([synthetic_df, sampled_df], axis=1)


synthetic_df = synthetic_df[df_impute_original.columns]

# Original dataset with missing values
data_iterative_random = df_impute_original.copy()

# Valid diseases per animal ('qualifying variables' that can be selected for ML imputation)
valid_diseases={}
for col in data_iterative_random.columns:
    
    #Ensuring good quality with a threshold (30+ countries must have data)
    if (data_iterative_random[col].notna().sum() >= 30):

        if ('_incidence' in col):
            parts = col.split('_')
            disease = '_'.join(parts[:-2])  
            animal_col = parts[-2]         #
            
    
            if animal_col not in valid_diseases:
                valid_diseases[animal_col] = []
            if disease not in valid_diseases[animal_col]:
                valid_diseases[animal_col].append(disease)
        
        if ('_vaccine_cov' in col):    
            parts = col.split('_')
            disease = '_'.join(parts[:-3]) 
            animal_col = parts[-3]         
    
            if animal_col not in valid_diseases:
                valid_diseases[animal_col] = []
            if disease not in valid_diseases[animal_col]:
                valid_diseases[animal_col].append(disease)



# A DataFrame to store the best hyperparameters for each column
hyperparams_table_iterative_random = pd.DataFrame()


# lists to store metrics
original_dataset_mae_scores = []
original_dataset_residuals_std_scores = []
expanded_dataset_mae_scores = []
expanded_dataset_residuals_std_scores = []

prediction_intervals_lower_bootstrap = []  # variability via bootstrpaping
prediction_intervals_upper_bootstrap = []  # 
prediction_intervals_lower_residuals = []  # variability via residuals
prediction_intervals_upper_residuals = []  # 
prediction_intervals_lower_asymmetric = []  # total variability
prediction_intervals_upper_asymmetric = []  #

diseases_flagged = {}



for column in data_iterative_random.columns:
    if (("_swine" in column) or ("_cattle" in column) or ("_poultry" in column)) and ('_incidence' in column):
       
        # Assign the appropriate population column
        if '_cattle' in column:
            pop_col = 'total cattle'
        elif '_poultry' in column:
            pop_col = 'total poultry'
        elif '_swine' in column:
            pop_col = 'total pigs'
            
        
        # Identify outliers and store them in the dictionary
        
        outlier_data= identify_outliers(data_iterative_random, column, pop_col)
        if len(outlier_data)>0:
            diseases_flagged[column]=outlier_data
            


#remove outliers
update_flagged_diseases(data_iterative_random, diseases_flagged)
update_flagged_diseases(synthetic_df, diseases_flagged) 

update_flagged_diseases(df_impute_sampling, diseases_flagged)


col_num = 0

tot_columns=0

for column in data_iterative_random.columns:
    if df_impute_original[df_impute_original[column].notna()]['ISO3'].nunique() >= 30:
        tot_columns+=1

for column in data_iterative_random.columns:
    
    if '_cattle' in column:
        weights = total_cattle_weights  # Use predicted values as weights (example)
        col_to_add = 'total cattle'
        col_to_add2='cattle_density'
        animal='cattle'
    elif '_poultry' in column:
        weights = total_poultry_weights  # Use predicted values as weights (example)
        col_to_add = 'total poultry'
        col_to_add2='poultry_density'
        animal='poultry'
    elif '_swine' in column:
        weights = total_swine_weights  # Use predicted values as weights (example)
        col_to_add = 'total pigs'
        col_to_add2= 'pig_density'
        animal='swine'
        
        
    if ("_swine" in column) or ("_cattle" in column) or ("_poultry" in column) :
        
        
        # If at least 30 countries w available data points
        if (
            (
                df_impute_original[df_impute_original[column].notna()]['ISO3'].nunique() >= 30
            ) 
         and df_impute_original[column].isnull().sum() > 0):
            col_num += 1
            
            print("Running imputation for ",column,'(',col_num,'/',tot_columns,')')

            
            
            
            countries_original=data_iterative_random['ISO3']
            original_X=data_iterative_random.drop(columns=[column, 'ISO3']).copy()
            original_y=data_iterative_random[column].copy()
            
            countries=synthetic_df['ISO3']
            X = synthetic_df.drop(columns=[column, 'ISO3'])  
            y = synthetic_df[column]  # The target column
            
            # Selecting the rows where the target is not missing for training
            X_train = X[~y.isnull()]
            y_train = y[~y.isnull()]
            countries = countries[~y.isnull()]
            
            mandatory_features = ["GDP per Capita", "Region_encoded", "prop_milk_cows", "prop_laying_chickens", "Year",col_to_add,'area_sq_km',col_to_add2]
            
            model = xgb.XGBRegressor(random_state=20, enable_categorical=True)

            
            other_animals = set(['cattle','swine','poultry'])-set(animal)

            n_samples_threshold = 30 * n_samples
            holder = X_train.copy()
            
            #Only keep pop sizes/densities of animals that can get infected with and spread same disease
            for other_animal in other_animals:
                #Check if '_vaccine_cov' and '_incidence' columns exist and meet criteria
                vaccine_col = column.replace('_incidence', '_vaccine_cov').replace('_' + animal, '_' + other_animal)
                incidence_col = column.replace('_vaccine_cov', '_incidence').replace('_' + animal, '_' + other_animal)

                vaccine_keep = (
                    vaccine_col in X_train.columns and 
                    X_train[vaccine_col].notna().sum() >= n_samples_threshold
                )
                incidence_keep = (
                    incidence_col in X_train.columns and 
                    X_train[incidence_col].notna().sum() >= n_samples_threshold
                )

                if not (vaccine_keep or incidence_keep):
                    # Set other_animal population columns to NaN
                    holder.loc[:, other_animal + "_density"] = np.nan
                    holder.loc[:, "total " + other_animal] = np.nan

            #Filter '_incidence' columns and set irrelevant ones to NaN
            #Bulding list of valid diseases
            
            #Only keep incidence for relevant diseases
            valid_diseases_for_animal = set(valid_diseases.get(animal, []))

            for col in X_train.columns:
                if '_incidence' in col:
                    parts = col.split('_')
                    disease = '_'.join(parts[:-2])
                    animal_col = parts[-2]

                    # Check conditions for setting to NaN
                    if animal_col != animal and disease not in valid_diseases_for_animal:
                        holder[col] = np.nan  # Use a single assignment instead of recreating a full array

                                    
            feature_importance = compute_feature_importance_group_cv(holder, y_train, countries, model, kf_splits=10)


            
            # Define and perform the Bayesian hyperparameter optimization for XGBoost
            #categorical enabled to account for the region encoding
            sampler = optuna.samplers.TPESampler(seed=2, n_startup_trials=100)
            study = optuna.create_study(sampler=sampler,direction='minimize', study_name=f"study_{column}")
            # Optimize the objective function
            study.optimize(
                lambda trial: objective(
                    trial, X_train, y_train, countries, feature_importance, mandatory_features
                ),
                n_trials=bayesian_trials,
                n_jobs=-1
            )
            
            

            # Get the best model and best parameters
            best_params = study.best_params
            
            prop_features=best_params.pop('feature_percentage')# Suggest the percentage of features to keep


            print(f'{bold_start}Optimal percent of total features: {prop_features*100:.2f}%{bold_end}')
            print(f'{bold_start}Optimal hyperparameters: {best_params}{bold_end}')
            

            best_model = xgb.XGBRegressor(**best_params,
                                          random_state=2,           
                                          enable_categorical=True,
                                          tree_method='hist'
                                         )    # Enable categorical handling
            
            best_params['feature_percentage']=prop_features
            

            # Continue with the rest of the feature selection process
            num_top_features = int(len(feature_importance) * prop_features)

            # Identify indices of mandatory features
            mandatory_indices = [i for i, col in enumerate(X_train.columns) if col in mandatory_features]

            # Exclude mandatory features from ranking
            non_mandatory_indices = [i for i in range(len(feature_importance)) if i not in mandatory_indices]

            # Rank non-mandatory features by importance
            sorted_non_mandatory_indices = [non_mandatory_indices[i] for i in np.argsort(feature_importance[non_mandatory_indices])[::-1]]

            # Select top-ranked non-mandatory features
            top_disease_feature_indices = sorted_non_mandatory_indices[:num_top_features]
            

            # Combine mandatory features with top-ranked non-mandatory features
            final_feature_indices = set(top_disease_feature_indices).union(mandatory_indices)

            # Map indices back to feature names
            top_features = [X_train.columns[i] for i in final_feature_indices]

            # Print selected features and their importance
            sorted_features = sorted(
                [(feature, feature_importance[X_train.columns.get_loc(feature)]) for feature in top_features],
                key=lambda x: x[1],
                reverse=True
            )
            
            print("Selected Features and Importance:")
            for feature, importance in sorted_features:
                print(f"Feature: {feature}, Importance: {importance:.4f}")

            
            # Update X_train with selected features
            X_train = X_train[top_features]
            original_X = original_X[top_features]
            X = X[top_features]
            
            
            best_model.fit(X_train, y_train) #now train on full dataset
            
            
            # Use the best model to predict missing values
            X_missing_original = original_X[original_y.isnull()]
            X_missing=X[y.isnull()]
            
            
            # Final prediction from the best model
            y_pred = best_model.predict(X_missing_original)
            y_pred = clip_values(y_pred,column)  # Ensure predictions stay between 0 and 1
            
            # Compute residuals and MAE based on the original dataset
            y_train_original = original_y[~original_y.isnull()]
            X_train_original = original_X[~original_y.isnull()]
            
            # Predict on the original dataset using the trained model
            y_train_pred_original = best_model.predict(X_train_original)
            
            # Calculate residuals and MAE for the original dataset
            residuals_original = y_train_original - y_train_pred_original
            mae_original = np.mean(np.abs(residuals_original))
            
                           
            #Print residual statistics
            print(f"Residuals for {column} - original dataset Mean: {residuals_original.mean()}, Std Dev: {residuals_original.std()}")
            
            
            #ADDED HERE: Compute residual variability for this column
            y_train_pred = best_model.predict(X_train)
            residuals = y_train - y_train_pred
            sigma_residual = residuals.std()
            
            # Asymmetric residuals
            sigma_residual_neg = residuals[residuals < 0].std()  # Made change: Capture under-predictions
            sigma_residual_pos = residuals[residuals > 0].std()  # Made change: Capture over-predictions
            
            
            # Generate predictions for the expanded dataset
            expanded_predictions = best_model.predict(X_train)  # Predict using the full expanded dataset
            
            # Calculate residuals only for rows with known values in the original dataset
            expanded_y = synthetic_df[column][~synthetic_df[column].isnull()]
            expanded_residuals = expanded_y[~synthetic_df[column].isnull()]-expanded_predictions
            # Calculate MAE for the expanded dataset
            expanded_mae = np.mean(np.abs(expanded_residuals))
            
            
            
            # Bootstrap to compute prediction intervals for the imputed values
            bootstrap_predictions = []
            
            for i in range(n_bootstrap):
                # Resample indices with replacement, using 100% of the data
                resampled_indices = resample(X_train.index, n_samples=int(1.0 * len(X_train)), replace=True)
                
                # Use these indices to select rows from X_train and y_train
                X_resampled = X_train.loc[resampled_indices]
                y_resampled = y_train.loc[resampled_indices]
                
                # Fit the model on the resampled data
                best_model.fit(X_resampled, y_resampled)
                
                # Predict on the missing values using the model trained on the resampled data
                y_pred_bootstrap = best_model.predict(X_missing_original)
                
                # Store the predictions from the current bootstrap iteration
                bootstrap_predictions.append(y_pred_bootstrap)
            
            # Convert to numpy array for easier computation
            bootstrap_predictions = np.array(bootstrap_predictions)
            
            #Compute variability in bootstrap predictions relative to the original model
            bootstrap_median = np.percentile(bootstrap_predictions, 50, axis=0)
            bootstrap_width_lower = bootstrap_median - np.percentile(bootstrap_predictions, 2.5, axis=0)
            bootstrap_width_upper = np.percentile(bootstrap_predictions, 97.5, axis=0) - bootstrap_median
            
            
            
            lower_bound_residuals=y_pred - 1.96 * sigma_residual_neg
            upper_bound_residuals= y_pred + 1.96 * sigma_residual_pos
            lower_bound_bootstrap=y_pred - bootstrap_width_lower
            upper_bound_bootstrap=y_pred + bootstrap_width_upper


            # Asymmetric Prediction Intervals
            lower_bound_asymmetric = y_pred - bootstrap_width_lower - 1.96 * sigma_residual_neg
            upper_bound_asymmetric = y_pred + bootstrap_width_upper + 1.96 * sigma_residual_pos
            
            print('lower_bound_asymmetric widths',np.abs(-bootstrap_width_lower - 1.96 * sigma_residual_neg))
            print('lower_bound_asymmetric widths',bootstrap_width_upper + 1.96 * sigma_residual_pos)

            

                
                
            mae_expanded_best=expanded_mae
            mae_original_best=mae_original
            residuals_std_best=residuals_original.std()
            expanded_dataset_residuals_std_scores_best=residuals.std()
            
            synthetic_df_col_best=synthetic_df[column]
            y_pred_best=y_pred
            synthetic_pred_best=clip_values(best_model.predict(X_missing),column)
            lower_residuals_best=lower_bound_residuals
            upper_residuals_best=upper_bound_residuals
            lower_bootstrap_best=lower_bound_bootstrap
            upper_bootstrap_best=upper_bound_bootstrap
            PI_low_asym_best=lower_bound_asymmetric
            PI_up_asym_best=upper_bound_asymmetric
            kept_best_params=best_params


            data_iterative_random.loc[original_y[original_y.isnull()].index, column] = y_pred_best
            
            #Fill in the synthetic_df column before predictions with the best sampled values 
            synthetic_df[column]=synthetic_df_col_best

            #Fill in synthetic df as well so that model iteratively "predicts better"
            synthetic_df.loc[y[y.isnull()].index, column] = synthetic_pred_best

            original_dataset_mae_scores.append(mae_original_best)
            original_dataset_residuals_std_scores.append(residuals_std_best)     
            #Save residual and MAE metrics for the expanded dataset
            expanded_dataset_mae_scores.append(mae_expanded_best)
            expanded_dataset_residuals_std_scores.append(expanded_dataset_residuals_std_scores_best)

            prediction_intervals_lower_bootstrap.append(lower_bootstrap_best) # variability via bootstrpaping
            prediction_intervals_upper_bootstrap.append(upper_bootstrap_best)  # 
            prediction_intervals_lower_residuals.append(lower_residuals_best)  # variability via residuals
            prediction_intervals_upper_residuals.append(upper_residuals_best)  # 
            prediction_intervals_lower_asymmetric.append(PI_low_asym_best)
            prediction_intervals_upper_asymmetric.append(PI_up_asym_best)
            # Store the best parameters in the hyperparams_table_iterative_random
            hyperparams_table_iterative_random[column] = pd.Series(kept_best_params)
                        
                        
# Transposing the hyperparams_table_iterative_random so that rows represent columns and columns represent hyperparameters
hyperparams_table_iterative_random = hyperparams_table_iterative_random.T

hyperparams_table_iterative_random['mean_absolute_error'] = expanded_dataset_mae_scores
hyperparams_table_iterative_random['residuals_std'] = expanded_dataset_residuals_std_scores

print("\nHyperparameter table:")
print(hyperparams_table_iterative_random)



In [45]:
len(prediction_intervals_lower_bootstrap)

91

In [46]:
countries_iso3s_dropdata_iterative_random = data_iterative_random.replace(-0.0, 0.0)
hyperparams_table_iterative_random = hyperparams_table_iterative_random.replace(-0.0, 0.0)

# Split into "_incidence"
df_incidence_hyperparams = hyperparams_table_iterative_random[
    hyperparams_table_iterative_random.index.str.endswith('_incidence')
].copy()

df_incidence_hyperparams.index=[i.replace('_swine','_pigs') for i in df_incidence_hyperparams.index]
df_incidence_hyperparams.rename(columns={'mean_absolute_error':'mean_absolute_error (Cases per 100,000 per Year)',
                                         'residuals_std':'std_residuals (Cases per 100,000 per Year)'},inplace=True)

# Split into "_vaccine_cov"
df_vaccine_cov_hyperparams = hyperparams_table_iterative_random[
    hyperparams_table_iterative_random.index.str.endswith('_vaccine_cov')
].copy()

df_vaccine_cov_hyperparams.index=[i.replace('_swine','_pigs') for i in df_vaccine_cov_hyperparams.index]

df_vaccine_cov_hyperparams['mean_absolute_error (Vaccination Coverage %)']=df_vaccine_cov_hyperparams['mean_absolute_error']*100
df_vaccine_cov_hyperparams['std_residuals (Vaccination Coverage %)']=df_vaccine_cov_hyperparams['residuals_std']*100
df_vaccine_cov_hyperparams.drop(columns=['mean_absolute_error','residuals_std'],inplace=True)

In [47]:
df_incidence_hyperparams.to_csv(os.path.join(notebook_dir,'Disease Incidence','Supplementary- Disease Incidence XGBoost Model Performance, Selected Hyperparameters.csv'),index=True, encoding='utf-8-sig')

df_vaccine_cov_hyperparams.to_csv(os.path.join(notebook_dir,'Vaccination Coverage','Supplementary- Vaccination Coverage XGBoost Model Performance, Selected Hyperparameters.csv'),index=True, encoding='utf-8-sig')

In [48]:
data_iterative_random_incidence=data_iterative_random.drop(columns=[i for i in data_iterative_random.columns if '_vaccine_cov' in i])
data_iterative_random_vaccine_cov=data_iterative_random.drop(columns=[i for i in data_iterative_random.columns if '_incidence' in i])

In [49]:
data_iterative_random_incidence.to_csv('incidence_all_imputed_XGBoost.csv',index=False,encoding='utf-8-sig')
data_iterative_random_vaccine_cov.to_csv('vaccine_cov_all_imputed_XGBoost.csv',index=False,encoding='utf-8-sig')

In [51]:
len(prediction_intervals_lower_residuals)

91

In [52]:
CI_data = []

# Extract relevant columns for swine, cattle, or poultry
relevant_columns = [column for column in df_impute_original.columns if ('_swine' in column) or ('_cattle' in column) or ('_poultry' in column)]
relevant_columns = [
    col for col in relevant_columns 
    if (df_impute_original[df_impute_original[col].notna()]['ISO3'].nunique() >= 30) 
]

# Loop through prediction intervals and columns
for CIs_disease_lower_residuals, CIs_disease_upper_residuals, \
    CIs_disease_lower_bootstrap, CIs_disease_upper_bootstrap, \
    CIs_disease_lower_asymmetric, CIs_disease_upper_asymmetric, \
    column in zip(prediction_intervals_lower_residuals,
                  prediction_intervals_upper_residuals,
                  prediction_intervals_lower_bootstrap,
                  prediction_intervals_upper_bootstrap,
                  prediction_intervals_lower_asymmetric,
                  prediction_intervals_upper_asymmetric,
                  relevant_columns):
    
    disease, animal = column.split('_', 1)

    animal = animal.title()
    
    # Loop through prediction intervals and corresponding ISO3 and Year
    for CI_lower_res, CI_upper_res, CI_lower_boot, CI_upper_boot, CI_lower_asym, CI_upper_asym, ISO3, year in zip(
        clip_values(CIs_disease_lower_residuals,column),
        clip_values(CIs_disease_upper_residuals,column),
        clip_values(CIs_disease_lower_bootstrap,column),
        clip_values(CIs_disease_upper_bootstrap,column),
        clip_values(CIs_disease_lower_asymmetric,column),
        clip_values(CIs_disease_upper_asymmetric,column),
        df_impute_original[df_impute_original[column].isna()]['ISO3'],
        df_impute_original[df_impute_original[column].isna()]['Year']):

        CI_data.append([
            ISO3, year, animal, disease, 
            CI_lower_res, CI_upper_res,
            CI_lower_boot, CI_upper_boot,
            CI_lower_asym, CI_upper_asym
        ])

CI_df = pd.DataFrame(CI_data, columns=[
    'ISO3', 'Year', 'Animal', 'Disease', 
    'Incidence Lower (Residual Only)', 'Incidence Upper (Residual Only)',
    'Incidence Lower (Bootstrap Only)', 'Incidence Upper (Bootstrap Only)',
    'Incidence Lower (Asymmetric)', 'Incidence Upper (Asymmetric)'
])

In [53]:
CI_df_incidence = CI_df[CI_df['Animal'].str.contains('_Incidence')].copy()
CI_df_incidence['Animal'] = CI_df_incidence['Animal'].str.replace('_Incidence', '', regex=False)


In [54]:
CI_df_vacc_cov = CI_df[CI_df['Animal'].str.contains('_Vaccine_Cov')].copy()
CI_df_vacc_cov['Animal'] = CI_df_vacc_cov['Animal'].str.replace('_Vaccine_Cov', '', regex=False)

CI_df_vacc_cov.columns=[i.replace('Incidence','Vaccination Coverage') for i in CI_df_vacc_cov.columns]

In [55]:
CI_df_incidence.to_csv('Incidence_CIs_XGBoost.csv',index=False,encoding='utf-8-sig')
CI_df_vacc_cov.to_csv('Vaccine_Coverage_CIs_XGBoost.csv',index=False,encoding='utf-8-sig')

# Generating dataframe to then append to original estimates (Incidence)

In [56]:
df_back= df_impute_original.drop(columns=['GDP per Capita',
 'prop_milk_cows',
 'prop_laying_chickens',
 'total pigs',
 'total poultry',
 'total cattle',
 'area_sq_km',
 'pig_density',
 'poultry_density',
 'cattle_density',
 'Region_encoded']+[i for i in df_impute_original.columns if '_vaccine_cov' in i])

df_melted = pd.melt(df_back, id_vars=['ISO3', 'Year'], var_name='disease_animal', value_name='value')

df_melted['Animal'] = df_melted['disease_animal'].apply(lambda x: x.split('_')[-1])
df_melted['Disease'] = df_melted['disease_animal'].apply(lambda x: '_'.join(x.split('_')[:-1]))

df_melted['Animal']=[i.title() for i in df_melted['Animal']]

# Pivot the dataframe to rearrange the diseases into separate columns
df_pivot = df_melted.pivot_table(index=['ISO3', 'Year', 'Animal'], columns='Disease', values='value').reset_index()
df_pivot.columns.name = None

# Melt the dataframe to have 'Disease' as a new column and corresponding 'Incidence' values
df_long = pd.melt(df_pivot, id_vars=['ISO3', 'Year', 'Animal'], var_name='Disease', value_name='Incidence')
df_long['Animal'] = df_long['Disease'].str.split('_').str[1].str.title()

df_long['Disease'] = df_long['Disease'].str.split('_').str[0]

df_original = df_long.dropna(subset=['Incidence'])

In [57]:
best_df=pd.read_csv('incidence_all_imputed_XGBoost.csv')
best_df.columns =[i.replace('_incidence','') for i in best_df.columns]

In [58]:
df_imputed_back=best_df.drop(columns=['GDP per Capita',
 'prop_milk_cows',
 'prop_laying_chickens',
 'total pigs',
 'total poultry',
 'total cattle',
 'area_sq_km',
 'pig_density',
 'poultry_density',
 'cattle_density',
 'Region_encoded'])

# Melt the dataframe to make 'animal' and disease columns
df_melted = pd.melt(df_imputed_back, id_vars=['ISO3', 'Year'], var_name='disease_animal', value_name='value')

df_melted['Animal'] = df_melted['disease_animal'].apply(lambda x: x.split('_')[-1])
df_melted['Disease'] = df_melted['disease_animal'].apply(lambda x: '_'.join(x.split('_')[:-1]))

df_melted['Animal']=[i.title() for i in df_melted['Animal']]

# Pivot the dataframe to rearrange the diseases into separate columns
df_pivot = df_melted.pivot_table(index=['ISO3', 'Year', 'Animal'], columns='Disease', values='value').reset_index()
df_pivot.columns.name = None

# Melting the dataframe to have 'Disease' as a new column and corresponding 'Incidence' values
df_long = pd.melt(df_pivot, id_vars=['ISO3', 'Year', 'Animal'], var_name='Disease', value_name='Incidence')
df_new = df_long.dropna(subset=['Incidence']).reset_index(drop=True)

In [59]:
# Drop the "Incidence" column from both dataframes for comparison
df1_compare = df_new.drop(columns=['Incidence']).reset_index(drop=True)
df2_compare = df_original.drop(columns=['Incidence'])

# Merge to identify rows that are common between dataframe1 and dataframe2, excluding "Incidence"
df_diff = pd.merge(df1_compare, df2_compare, how='left', indicator=True)


# Boolean indexing to keep rows present only in df_new (i.e., 'left_only')
df_filtered = df_new.loc[df_diff['_merge'] == 'left_only'].copy()

# Add a new column 'Source' with the value 'Imputed'
df_filtered['Source'] = 'Imputed'

In [60]:
very_final_df=pd.merge(df_filtered, CI_df_incidence, how='left', indicator=False, on=['ISO3','Year','Animal','Disease'])

In [61]:
very_final_df.rename(columns={'Incidence':'Incidence (Cases per 100,000)',
                             'Incidence Upper': 'Incidence (Cases per 100,000) Upper',
                             'Incidence Lower': 'Incidence (Cases per 100,000) Lower'},inplace=True)
very_final_df.to_csv(os.path.join(notebook_dir,'Disease Incidence',"FINAL_ALL_incidence_imputations_to_add.csv"),index=False,encoding='utf-8-sig')

# Generating dataframe to then append to original estimates (Vaccination Coverage)

In [62]:
df_back= df_impute_original.drop(columns=['GDP per Capita',
 'prop_milk_cows',
 'prop_laying_chickens',
 'total pigs',
 'total poultry',
 'total cattle',
 'area_sq_km',
 'pig_density',
 'poultry_density',
 'cattle_density',
 'Region_encoded']+[i for i in df_impute_original.columns if '_incidence' in i])

# Melt the dataframe to make 'animal' and disease columns
df_melted = pd.melt(df_back, id_vars=['ISO3', 'Year'], var_name='disease_animal', value_name='value')

# Extract 'animal' and 'disease' from 'disease_animal' column
df_melted['Animal'] = df_melted['disease_animal'].apply(lambda x: x.split('_')[1])
df_melted['Disease'] = df_melted['disease_animal'].apply(lambda x: x.split('_')[0])

df_melted['Animal']=[i.title() for i in df_melted['Animal']]

# Pivot the dataframe to rearrange the diseases into separate columns
df_pivot = df_melted.pivot_table(index=['ISO3', 'Year', 'Animal'], columns='Disease', values='value').reset_index()
df_pivot.columns.name = None

# Melting the dataframe to have 'Disease' as a new column and corresponding 'Incidence' values
df_long = pd.melt(df_pivot, id_vars=['ISO3', 'Year', 'Animal'], var_name='Disease', value_name='Vaccination Coverage')
df_original = df_long.dropna(subset=['Vaccination Coverage'])

In [63]:
best_df=pd.read_csv('vaccine_cov_all_imputed_XGBoost.csv')
best_df.columns =[i.replace('_vaccine_cov','') for i in best_df.columns]

In [64]:
df_imputed_back=best_df.drop(columns=['GDP per Capita',
 'prop_milk_cows',
 'prop_laying_chickens',
 'total pigs',
 'total poultry',
 'total cattle',
 'area_sq_km',
 'pig_density',
 'poultry_density',
 'cattle_density',
 'Region_encoded'])

# Melt the dataframe to make 'animal' and disease columns
df_melted = pd.melt(df_imputed_back, id_vars=['ISO3', 'Year'], var_name='disease_animal', value_name='value')

# Extract 'animal' and 'disease' from 'disease_animal' column
df_melted['Animal'] = df_melted['disease_animal'].apply(lambda x: x.split('_')[1])
df_melted['Disease'] = df_melted['disease_animal'].apply(lambda x: x.split('_')[0])

df_melted['Animal']=[i.title() for i in df_melted['Animal']]

# Pivot the dataframe to rearrange the diseases into separate columns
df_pivot = df_melted.pivot_table(index=['ISO3', 'Year', 'Animal'], columns='Disease', values='value').reset_index()
df_pivot.columns.name = None

# Melt the dataframe to have 'Disease' as a new column and corresponding 'Vaccination coverage' values
df_long = pd.melt(df_pivot, id_vars=['ISO3', 'Year', 'Animal'], var_name='Disease', value_name='Vaccination Coverage')
df_new = df_long.dropna(subset=['Vaccination Coverage']).reset_index(drop=True)

In [65]:
# Drop the "Vaccination coverage" column from both dataframes for comparison
df1_compare = df_new.drop(columns=['Vaccination Coverage']).reset_index(drop=True)
df2_compare = df_original.drop(columns=['Vaccination Coverage']).reset_index(drop=True)

# Merge to identify rows that are common between dataframe1 and dataframe2, excluding "Vaccination coverage"
df_diff = pd.merge(df1_compare, df2_compare, how='left', indicator=True)

In [66]:
# Drop "Vaccination coverage" from both dataframes for comparison
df1_compare = df_new.drop(columns=['Vaccination Coverage']).reset_index(drop=True)
df2_compare = df_original.drop(columns=['Vaccination Coverage'])

# Merge  to identify rows that are common between dataframe1 and dataframe2, excluding "Vaccination coverage"
df_diff = pd.merge(df1_compare, df2_compare, how='left', indicator=True)


# Now boolean indexing to keep rows present only in df_new (i.e., 'left_only')
df_filtered = df_new.loc[df_diff['_merge'] == 'left_only'].copy()

# Add a new column 'Source' with the value 'Imputed'
df_filtered['Source'] = 'Imputed'


In [67]:
very_final_df=pd.merge(df_filtered, CI_df_vacc_cov, how='left', indicator=False, on=['ISO3','Year','Animal','Disease'])

In [68]:
#USA does not vaccinate against the velogenic strain of newcastle disease (just non-velogenic)
very_final_df=very_final_df[~((very_final_df['ISO3']=='USA')&(very_final_df['Disease']=='Newcastle disease (velogenic)'))]

In [69]:
very_final_df.to_csv(os.path.join(notebook_dir,'Vaccination Coverage','FINAL_ALL_vaccine_cov_imputations_to_add.csv'),index=False,encoding='utf-8-sig')